In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
# linear regression using ordinary least squares 
def linear_regression(X_in, Y):  
    X = np.ones((len(X_in), 2), np.uint64)
    X[:, 1] = X_in
    Xt = X.transpose()
    
    # compute beta = (X^t . X)^-1 . X^t . Y
    coefficients = ((np.linalg.inv(Xt.dot(X))).dot(Xt)).dot(Y)
    coefficients[0], coefficients[1] = coefficients[1], coefficients[0]
    
    return coefficients

# format defence and hitpoint experience
def process_df(df):
    exp_type = np.zeros(len(df))
    
    d1 = { "Time (min)" : df["Time (min)"],
           "Exp Gained" : df["Def Exp Gained"],
           "Exp Type"   : exp_type } # initialize exp type column
    defence_df = pd.DataFrame(data = d1)
    defence_df.iloc[:, 2] = "defence" # declare exp type column

    d2 = { "Time (min)" : df["Time (min)"],
           "Exp Gained" : df["HP Exp Gained"],
           "Exp Type"   : exp_type }
    hitpoint_df = pd.DataFrame(data = d2)
    hitpoint_df.iloc[:, 2] = "hitpoint"

    processed_df = defence_df.append(hitpoint_df)

    return processed_df

# return the Y values needed for linear regression 
def get_linreg_Y(df):
    category_length = int(len(df) / 2)
    Y = df.iloc[category_length:, 1].values
    
    return Y

# return a dataframe containing copies of dataframes 1 and 2
# the Event RPG dataframe must go first
def combine_df(df1, df2):
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    
    df1_copy["Weapon Type"], df2_copy["Weapon Type"] = np.zeros(len(df1)), np.zeros(len(df1))
    df1_copy.loc[:, "Weapon Type"] = "Event RPG"
    df2_copy.loc[:, "Weapon Type"] = "Iron Scimitar"
    
    combined_df = df1_copy.append(df2_copy)
    
    return combined_df  

# calculate the number of hours required to reach final_exp given the initial_exp and coefficients
def extrapolate_time(initial_exp, final_exp, coefficients):
    exp_required = final_exp - initial_exp
    
    time = (exp_required - coefficients[1]) / coefficients[0] / 60
    
    return time

In [9]:
# load preprocessed data
rpg_df = pd.read_csv("data/rpg_data.csv")
scim_df = pd.read_csv("data/scim_data.csv")

In [ ]:
# process data
processed_rpg_df = process_df(rpg_df)
processed_scim_df = process_df(scim_df)
combined_df = combine_df(processed_rpg_df, processed_scim_df)

In [ ]:
# write processed data to csv
processed_rpg_df.to_csv("data/processed_rpg_data.csv", index = False)
processed_scim_df.to_csv("data/processed_scim_data.csv", index = False)
combined_df.to_csv("data/combined_df.csv", index = False)

In [ ]:
# load processed data (skip loading, processing and writing preprocessed data)
processed_rpg_df = pd.read_csv("data/processed_rpg_data.csv")
processed_scim_df = pd.read_csv("../data/processed_scim_data.csv")
combined_df = pd.read_csv("../data/combined_df.csv")

In [ ]:
# transform data for linear regression
category_length = int(len(processed_rpg_df) / 2)
time = np.arange(0, 190, 10)

rpg_defence_exp, rpg_hitpoint_exp = processed_rpg_df.iloc[:category_length, 1], processed_rpg_df.iloc[category_length:, 1]
scim_defence_exp, scim_hitpoint_exp = processed_scim_df.iloc[:category_length, 1], processed_scim_df.iloc[category_length:, 1]

In [ ]:
# linear regression coefficients for the Event RPG
print("--- Event RPG ---")
rpg_defence_coeffs = linear_regression(time, rpg_defence_exp.values)
rpg_defence_stats_coeffs = stats.linregress(time, rpg_defence_exp.values)
print("Defence EXP")
print("Slope and x-intercept using matricies: ", rpg_defence_coeffs)
print("Slope and x-intercept using SciPy stats: ", rpg_defence_stats_coeffs[0:2])

rpg_hitpoint_coeffs = linear_regression(time, rpg_hitpoint_exp.values)
rpg_hitpoint_stats_coeffs = stats.linregress(time, rpg_hitpoint_exp.values)
print("\nHitpoint EXP")
print("Slope and x-intercept using matricies: ", rpg_hitpoint_coeffs)
print("Slope and x-intercept using SciPy stats: ", rpg_hitpoint_stats_coeffs[0:2])

In [ ]:
# linear regression coefficients for the iron scimitar
print("--- Iron Scimitar ---")
scim_defence_coeffs = linear_regression(time, scim_defence_exp.values)
scim_defence_stats_coeffs = stats.linregress(time, scim_defence_exp.values)
print("Defence EXP")
print("Slope and x-intercept using matricies: ", scim_defence_coeffs)
print("Slope and x-intercept using SciPy stats: ", scim_defence_stats_coeffs[0:2])

scim_hitpoint_coeffs = linear_regression(time, scim_hitpoint_exp.values)
scim_hitpoint_stats_coeffs = stats.linregress(time, scim_hitpoint_exp.values)
print("\nHitpoint EXP")
print("Slope and x-intercept using matricies: ", scim_hitpoint_coeffs)
print("Slope and x-intercept using SciPy stats: ", scim_hitpoint_stats_coeffs[0:2])

In [ ]:
# visualize data
exp_scatter = sns.relplot(x = "Time (min)", 
                          y = "Exp Gained",
                          hue = "Exp Type", 
                          col = "Weapon Type",
                          data = combined_df)

exp_scatter.set(xlabel = "Time (min)",
                xticks = processed_rpg_df.iloc[1:len(combined_df) // 4:2, 0],
                ylabel = "Experience",
                yticks = range(0, 5000, 500))

plt.show()

In [ ]:
# extrapolate to obtain the amount of time required to reach level 75 defence (1210421 exp) from 133685 exp
rpg_time = round(extrapolate_time(133685, 1210421, rpg_defence_coeffs))
print("Hours required to reach level 75 Defence using the Event RPG:", rpg_time)

scim_time = round(extrapolate_time(133685, 1210421, scim_defence_coeffs))
print("Hours required to reach level 75 Defence using the Event RPG:", scim_time)